### Import Required Libraries

We import the core Python libraries required for data manipulation, path handling, and exploratory analysis.

In [1]:
import pandas as pd
from pathlib import Path


### Loading the Training and Test Datasets

The training and test datasets are loaded from the `data/` directory. Paths are resolved explicitly to avoid issues with relative locations.
These datasets will be used for exploratory data analysis and feature engineering.

In [2]:

train_path = Path("../data/train.csv").resolve()

out_dir = Path("../data/processed")
out_dir.mkdir(parents=True, exist_ok=True)

train_out_path = out_dir / "train_cleaned.csv"



In [3]:
train_data = pd.read_csv(train_path)


### Previewing the Training Data

A quick preview helps verify that the data has loaded correctly and provides an initial sense of feature types and values.


In [ ]:
train_data.head()

### Inspecting Dataset Structure and Data Types

The info() method provides a concise summary of the dataset, including:

1. Number of entries

2. Column names

3. Data types

4. Count of non-null values

This is especially useful for identifying missing values.

In [ ]:
train_data.info()

### Statistical Summary of Numerical Features

The describe() method generates descriptive statistics for numerical columns, such as:

1. Mean

2. Standard deviation

3. Minimum and maximum values

4. Quartiles

This gives an overview of data distribution and potential anomalies.

In [ ]:
train_data.describe()

### Identifying Missing Age Values

Here we filter the dataset to show only rows where the Age column has missing values (NaN).
This helps us understand how many records are affected and plan an imputation strategy.

In [ ]:
train_data[train_data['Age'].isna()]

### Extract Passenger Titles from Names

Passenger titles (e.g., Mr, Mrs, Miss) are extracted from the Name column using a regular expression.
Titles often capture social status and correlate strongly with age and survival.

In [4]:
train_data['Title'] = train_data['Name'].str.extract(r',\s*([^\.]+)\.')


### Grouping Rare Titles

Many titles appear very infrequently.
To reduce noise and dimensionality, we group uncommon titles into a single category called "Rare".

In [5]:
train_data['Title'] = train_data['Title'].where(
train_data['Title'].isin(['Mr', 'Mrs', 'Master', 'Miss']),
'Rare'
)


### Encoding Titles as Numerical Values

Machine learning models require numerical inputs.
Here we map each title category to an integer value for model compatibility.

### Impute Missing Age Values Using Training Data Only

Missing values in the Age column are filled using the average age per title, calculated only from the training data.
This avoids data leakage while producing more realistic age estimates.

In [9]:
title_age_means = (
    train_data
    .groupby('Title')['Age']
    .mean()
)

for title, avg_age in title_age_means.items():
    train_data.loc[
        (train_data['Title'] == title) & (train_data['Age'].isna()),
        'Age'
    ] = round(avg_age, 2)



### Validating the Sex Column

We check whether the Sex column contains only the expected values (male, female).
This ensures data consistency before encoding.

In [10]:
train_data[~train_data['Sex'].isin(['male', 'female'])]


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title


### Encoding Sex as Numerical Values

We convert the categorical Sex column into numeric form:

male → 0

female → 1

This is required for machine learning algorithms.

In [11]:
train_data['Sex'] = train_data['Sex'].map({
    'male': 0,
    'female': 1
})


### Exploring the Embarked Feature

We inspect the Embarked column, which represents the port where passengers boarded the ship.

In [12]:
train_data['Embarked']

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: str

### Validating Embarked Values

This step checks for unexpected or invalid embarkation values outside the known categories:

S (Southampton)

C (Cherbourg)

Q (Queenstown)

In [13]:
train_data[~train_data['Embarked'].isin(['S','C','Q'])]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
61,62,1,1,"Icard, Miss. Amelie",1,38.0,0,0,113572,80.0,B28,NaN,Miss
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",1,62.0,0,0,113572,80.0,B28,NaN,Mrs


### Handling Missing Embarked Values

Missing values in Embarked are filled with 'C', which is the most frequent embarkation port in this dataset.
This ensures no missing values remain in this feature.

In [14]:
train_data['Embarked'] = train_data['Embarked'].fillna('C')


### Final Data Validation

As a final check, we confirm that there are no unexpected missing values remaining in either dataset.

In [ ]:
print("Train missing values:")
print(train_data.isna().sum())


### Export Cleaned Datasets to CSV Files

The cleaned datasets are saved as CSV files for use in modeling and experimentation.

In [15]:
train_data.to_csv(train_out_path, index=False)

